# Documentation: Data base managers

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from astropy.coordinates import Angle
import astropy.units as u
import os
from pandas import DataFrame
import platform

import constraints as c
from db import DBCreator, TelescopeManager, FieldManager, GuidestarManager, ObservationManager
from fieldgrid import FieldGridIsoLat

## `DBCreator`: create survey planner database

The `DBCreator` class provides the basic functionalities to create a new database:

In [3]:
db_name = 'test_planner_init.sqlite3'
db = DBCreator(db_name)

### Create database

In [4]:
created = db.create()

Database file exists. Overwrite (y) or cancel (enter)? y


Database 'test_planner_init.sqlite3' created.
Table 'Fields' created.
Table 'Telescopes' created.
Table 'ParameterSets' created.
Table 'Constraints' created.
Table 'Parameters' created.
Table 'ParameterNames' created.
Table 'Observability' created.
Table 'ObservabilityStatus' created.
Table 'ObsWindows' created.
Table 'TimeRanges' created.
Table 'Observations' created.
Table 'Guidestars' created.
Table 'Filters' created.
Constraints added to table 'Constraints'.
Statuses added to table 'ObservabilityStatus'.
Database creation finished.

Note: Next you need to add observatories, constraints, fields, guidestars, and observations.


### Add telescopes

In [5]:
if created:
    name = 'Skinakas'
    lat = Angle('35:12:43 deg')
    lon = Angle('24:53:57 deg')
    height = 1750.
    utc_offset = 2.
    db.add_telescope(name, lat.rad, lon.rad, height, utc_offset)

Telescope 'Skinakas' added.


In [6]:
if created:
    name = 'SAAO'
    lat = Angle('-32:22:46 deg')
    lon = Angle('20:48:38.5 deg')
    height = 1798.
    utc_offset = 2.
    db.add_telescope(name, lat.rad, lon.rad, height, utc_offset)

Telescope 'SAAO' added.


It is not possible to add two telescopes with the same name:

In [7]:
if created:
    name = 'SAAO'
    lat = Angle('-32:22:46 deg')
    lon = Angle('20:48:38.5 deg')
    height = 1798.
    utc_offset = 2.
    db.add_telescope(name, lat.rad, lon.rad, height, utc_offset)

Telescope 'SAAO' already exists. Name needs to be unique.


### Add constraints

We add constraints to each telescope separately. Some constraints are the same, some are telescope specific.

In [8]:
if created:
    # general limits:
    twilight = 'nautical'
    airmass_limit = c.AirmassLimit(2.)
    moon_distance = c.MoonDistance(10.)
    
    # Skinakas specific limits:
    hourangle_limit = c.HourangleLimit(5.33)
    
    # SAAO specific limits:
    polygon_ha = [-4, -4, 0.8, 0.8, 1.7, 2.4, 3.2, 2.8]
    polygon_dec = [0.1, -52, -65, -85, -74.5, -70.0, -60, 0.1]
    ha_dec_limit = c.PolyHADecLimit(polygon_ha, polygon_dec)
    
    db.add_constraints('Skinakas', twilight, constraints=(airmass_limit, hourangle_limit, moon_distance))
    db.add_constraints('SAAO', twilight, constraints=(airmass_limit, moon_distance, ha_dec_limit))

Constraint 'AirmassLimit' for telescope 'Skinakas'.
Constraint 'HourangleLimit' for telescope 'Skinakas'.
Constraint 'MoonDistance' for telescope 'Skinakas'.
Constraint 'AirmassLimit' for telescope 'SAAO'.
Constraint 'MoonDistance' for telescope 'SAAO'.
Constraint 'PolyHADecLimit' for telescope 'SAAO'.


It is not possible to add constraints to a non-existing telescope. Thus, telescopes need to be added first.

In [9]:
db.add_constraints('does-not-exist', twilight, constraints=(airmass_limit, hourangle_limit, moon_distance))

### Add fields

First, we create the field grid using the `FieldGridIsoLat` class. For a quick demonstration we do not create the actual Pasiphae grid, but a coarser grid with fewer fields.

Add Northern fields:

In [10]:
fov = Angle(5 * u.deg)
overlap_ns = Angle(fov / 2.)
overlap_ew = Angle(1 * u.deg)
tilt = Angle(0 * u.deg)
dec_lim_north = Angle(90 * u.deg)
dec_lim_south = Angle(0 * u.deg)
gal_lat_lim = Angle(30 * u.deg)
gal_lat_lim_strict = True
verbose = 1

grid = FieldGridIsoLat(
        fov.rad, overlap_ns=overlap_ns.rad, overlap_ew=overlap_ew.rad, tilt=tilt.rad, dec_lim_north=dec_lim_north.rad, 
        dec_lim_south=dec_lim_south.rad, gal_lat_lim=gal_lat_lim.rad, gal_lat_lim_strict=gal_lat_lim_strict, verbose=verbose)

Create fields..
  Calculate field centers..
  Calculate field corners..
    Done                                                    
  Identify fields in Galactic plane..
Final number of fields: 1097


In [11]:
if created:
    db.add_fields(grid, 'Skinakas', active=True)

1097 fields added to database.                   


Add Southern fields:

In [12]:
fov = Angle(7 * u.deg)
overlap_ns = Angle(fov / 2.)
overlap_ew = Angle(1 * u.deg)
tilt = Angle(0 * u.deg)
dec_lim_north = Angle(0 * u.deg)
dec_lim_south = Angle(-90 * u.deg)
gal_lat_lim = Angle(30 * u.deg)
gal_lat_lim_strict = True
verbose = 1

grid = FieldGridIsoLat(
        fov.rad, overlap_ns=overlap_ns.rad, overlap_ew=overlap_ew.rad, tilt=tilt.rad, dec_lim_north=dec_lim_north.rad, 
        dec_lim_south=dec_lim_south.rad, gal_lat_lim=gal_lat_lim.rad, gal_lat_lim_strict=gal_lat_lim_strict, verbose=verbose)

Create fields..
  Calculate field centers..
  Calculate field corners..
    Done                                                    
  Identify fields in Galactic plane..
Final number of fields: 547


In [13]:
if created:
    db.add_fields(grid, 'SAAO', active=True)

547 fields added to database.                   


### Add guidestars

For a simple demonstration we claim there is a guide star at each field center, i.e. we use the field centers as coordinates for the guide stars.

In [14]:
# get fields:
manager = FieldManager(db_name)
field_ids = []
field_center_ras = []
field_center_decs = []

for field in manager.get_fields():
    field_ids.append(field['field_id'])
    field_center_ras.append(field['center_ra'])
    field_center_decs.append(field['center_dec'])

We are going to add guidestars for all but the last to fields. At the end we will be warned that there are fields without associated guidestars.

In [15]:
db.add_guidestars(field_ids[:-2], field_center_ras[:-2], field_center_decs[:-2], warn_missing=True)

1642 new guidestars added to database.



By setting `warn_rep`, a lower limit that guidestars for the same field need to be separated, we can also get a warning when we try to add a new guidestar that is considered a duplicate of one already stored:

In [16]:
db.add_guidestars(field_ids[0], field_center_ras[0], field_center_decs[0], warn_rep=Angle(2*u.arcmin), warn_missing=False)

New guidestar
  field ID: 1
  RA:   00d00m00s
  Dec: +02d30m00s
is close to the following stored guidestar(s):
1. Guidestar ID: 1
  RA:   00d00m00s
   Dec: +02d30m00s
   separation: 0.0000 deg


Add this new guidestar anyway? (y/n)  n


0 new guidestars added to database.


As a final check, we can also set `warn_sep`, a maximum limit by which the guidestar may be separated from the corresponding field center. We get a warning if a guidestart is too far off:

In [17]:
db.add_guidestars(
        field_ids[-1], field_center_ras[-1]+Angle(18*u.arcmin).rad, field_center_decs[-1],
        warn_sep=Angle(15*u.arcmin), warn_missing=False)

New guide star 0 for field ID 1644 is too far from the field center with separation 0d17m58.51989416s.


Add it to the database anyway? (y/n)  n


0 new guidestars added to database.


### Add observations

We can simply add the same observation to each field. The filter does not exist in the new database. We are automatically asked, whether or not to add it.

In [18]:
db.add_observations(30., 2, 'r')

Filter 'r does not exist. Add it to data base? (y/n) y


1644 observations added to data base.


If at least one observation with the same parameters exist for a field and has not been finished yet, we automatically get notified that we are trying to add a duplicate:

In [19]:
db.add_observations(30., 2, 'r')

1 observation(s) with the same parameters already exist in data base. 0 out of those are finished. Add new observation anyway? (y/n, 'ALL' to add all following without asking, or 'NONE' to skip all existing observations that have not been finished). n
1 observation(s) with the same parameters already exist in data base. 0 out of those are finished. Add new observation anyway? (y/n, 'ALL' to add all following without asking, or 'NONE' to skip all existing observations that have not been finished). NONE


0 observations added to data base.


### Database info

<div class="alert alert-block alert-warning">

- Add dbinfo() method to DBManager that prints out general information about the database. Include it here in the documentation.

</div>

## Database managers

**Important:** Any changes to the database should be made via these managers. The database should not be edited manually, because that may severly interfere with the survey planning.

For a demonstration of the managers we copy the database that was created above and work with the copy in the following.

In [20]:
db_init = 'test_planner_init.sqlite3'
db_name = 'test_planner_temp.sqlite3'

if platform.system() == 'Linux':
    os.system(f'cp {db_init} {db_name}')
elif platform.system() == 'Windows':
    os.system(f'xcopy {db_init} {db_name} /y')
else:
    raise ValueError('Unknown operating system.')

### `TelescopeManager`

The `TelescopeManager` class provides the methods to add telescopes and constraints to the database and read them from the database.
Below, we describe the methods relevant for user interaction. Other methods are used by the survey planning software.

#### Add telescope

In [40]:
db_name = 'test_planner_temp.sqlite3'
manager = TelescopeManager(db_name)

We can use the same method as in the `DBCreator` class to add a telescope. Duplications are not allowed:

In [22]:
name = 'Skinakas'
lat = Angle('35:12:43 deg')
lon = Angle('24:53:57 deg')
height = 1750.
utc_offset = 2.
manager.add_telescope(name, lat.rad, lon.rad, height, utc_offset)

Telescope 'Skinakas' already exists. Name needs to be unique.


**Note:** There is not way to change a telescope. Changing its parameters would require that all observabilities need to be re-calculated. Therefore, it makes sense to start with a new database.

#### Add/change constraints

We can add constraints as we did with the `DBCreator` class. If the telescope already has constraints associated we get a warning:

In [23]:
# general limits:
twilight = 'nautical'
airmass_limit = c.AirmassLimit(2.)
moon_distance = c.MoonDistance(10.)

# Skinakas specific limits:
hourangle_limit = c.HourangleLimit(5.33)

manager.add_constraints('Skinakas', twilight, constraints=(airmass_limit, hourangle_limit, moon_distance))

**Note:** The method does not check whether or not these new constraints differ in any way from the already stored ones. Above we try to add the exact same constraints we already added. This would be unwise to do, because it would invalidate all observabilities that were already calculated, even though they would technically still be valid as the constraints are the same.

**Note:** Adding a new set of constraints, means that we need to define all constraints that should from now one be associated with the telescope. We cannot add a single constraint to the ones existing already in the database. 

Let's say we want to decrease the Moon separation limit:

In [24]:
# general limits:
twilight = 'nautical'
airmass_limit = c.AirmassLimit(2.)
moon_distance = c.MoonDistance(5.)

# Skinakas specific limits:
hourangle_limit = c.HourangleLimit(5.33)

manager.add_constraints('Skinakas', twilight, constraints=(airmass_limit, hourangle_limit, moon_distance))

Parameter set with ID 1 deactivated.
0 corresponding observabilities deactivated.
0 corresponding observing windows deactivated.
0 corresponding time ranges deactivated.
Constraint 'AirmassLimit' for telescope 'Skinakas'.
Constraint 'HourangleLimit' for telescope 'Skinakas'.
Constraint 'MoonDistance' for telescope 'Skinakas'.


**Note:** Constraints must always be changed this way. Only this way ensures that the observabilities are calculated anew. This would not be the case with manual changes to the database.

#### Get telescopes

`get_telescopes()` gives us **all stored telescopes** in a list. Each list item is a dict that contains the telescope parameters:

In [25]:
db_name = 'test_planner_temp.sqlite3'
DataFrame(manager.get_telescopes())

,telescope_id,name,lat,lon,height,utc_offset
0,1,Skinakas,0.614564,0.434572,1750.0,2.0
1,2,SAAO,-0.565128,0.363215,1798.0,2.0


We can **specify a telescope name** to get just that telescope:

In [26]:
DataFrame(manager.get_telescopes(name='Skinakas'))

,telescope_id,name,lat,lon,height,utc_offset
0,1,Skinakas,0.614564,0.434572,1750.0,2.0


We can also **include the active constraints** that are associated with the telescope, by setting `constraints=True`:

In [41]:
manager.get_telescopes(name='Skinakas', constraints=True)

[{'telescope_id': 1,
  'name': 'Skinakas',
  'lat': 0.6145643665848811,
  'lon': 0.43457243933615475,
  'height': 1750.0,
  'utc_offset': 2.0,
  'parameter_set_id': 3,
  'constraints': {'Twilight': {'twilight': -12.0},
   'AirmassLimit': {'limit': 2.0, 'conversion': 'secz'},
   'HourangleLimit': {'limit': 5.33, 'limit_lo': -5.33},
   'MoonDistance': {'limit': 5.0}}}]

If we want to see all associated constraints, regardless of whether they are active or not, we use `constraints='all'`:

In [7]:
manager.get_telescopes(name=None, constraints='all')

[{'telescope_id': 1,
  'name': 'Skinakas',
  'lat': 0.6145643665848811,
  'lon': 0.43457243933615475,
  'height': 1750.0,
  'utc_offset': 2.0,
  'parameter_sets': [{'parameter_set_id': 1,
    'telescope': 'Skinakas',
    'active': False,
    'constraints': {'Twilight': {'twilight': -12.0},
     'AirmassLimit': {'limit': 2.0, 'conversion': 'secz'},
     'HourangleLimit': {'limit': 5.33, 'limit_lo': -5.33},
     'MoonDistance': {'limit': 10.0}}},
   {'parameter_set_id': 3,
    'telescope': 'Skinakas',
    'active': True,
    'constraints': {'Twilight': {'twilight': -12.0},
     'AirmassLimit': {'limit': 2.0, 'conversion': 'secz'},
     'HourangleLimit': {'limit': 5.33, 'limit_lo': -5.33},
     'MoonDistance': {'limit': 5.0}}}]},
 {'telescope_id': 2,
  'name': 'SAAO',
  'lat': -0.5651279155221417,
  'lon': 0.3632151376820477,
  'height': 1798.0,
  'utc_offset': 2.0,
  'parameter_sets': [{'parameter_set_id': 2,
    'telescope': 'SAAO',
    'active': True,
    'constraints': {'Twilight': {'

#### Get constraints

In `get_constraints()` we can specify a telescope, if we want to get only the constraints associated with that telescope. With `active=True` we only get active constraints. Otherwise, we get active and inactive constraints. 

If we specify the telescope and set `active=True` there is only one set of constraints, because only one set can be active for a telescope. 
Otherwise, we may get multiple sets as a `list`. Sets contain all the constraints that were added to a telescope via one call of the `add_constraints()` method.

Here, we are calling for **one specific telescope and its active constraints**:

In [8]:
manager.get_constraints(telescope='Skinakas', active=True)

[{'parameter_set_id': 3,
  'telescope': 'Skinakas',
  'active': True,
  'constraints': {'Twilight': {'twilight': -12.0},
   'AirmassLimit': {'limit': 2.0, 'conversion': 'secz'},
   'HourangleLimit': {'limit': 5.33, 'limit_lo': -5.33},
   'MoonDistance': {'limit': 5.0}}}]

We can get an overview over **all existing constraints** like this:

In [9]:
manager.get_constraints(telescope=None, active=False)

[{'parameter_set_id': 1,
  'telescope': 'Skinakas',
  'active': False,
  'constraints': {'Twilight': {'twilight': -12.0},
   'AirmassLimit': {'limit': 2.0, 'conversion': 'secz'},
   'HourangleLimit': {'limit': 5.33, 'limit_lo': -5.33},
   'MoonDistance': {'limit': 10.0}}},
 {'parameter_set_id': 2,
  'telescope': 'SAAO',
  'active': True,
  'constraints': {'Twilight': {'twilight': -12.0},
   'AirmassLimit': {'limit': 2.0, 'conversion': 'secz'},
   'MoonDistance': {'limit': 10.0},
   'PolyHADecLimit': {'ha': [-4.0, -4.0, 0.8, 0.8, 1.7, 2.4, 3.2, 2.8],
    'dec': [0.1, -52.0, -65.0, -85.0, -74.5, -70.0, -60.0, 0.1]}}},
 {'parameter_set_id': 3,
  'telescope': 'Skinakas',
  'active': True,
  'constraints': {'Twilight': {'twilight': -12.0},
   'AirmassLimit': {'limit': 2.0, 'conversion': 'secz'},
   'HourangleLimit': {'limit': 5.33, 'limit_lo': -5.33},
   'MoonDistance': {'limit': 5.0}}}]

#### Print info about telescopes and constraints

We can use the `info()` method to learn about what telescopes are stored in the database. If we use `constraints=True`, we additionally get info about the currently active, associated constraints. With `constraints='all'` we learn about all associated parameter sets, including the inactive ones:

In [11]:
manager.info(constraints='all')

============ TELECOPES ============
2 telescopes stored in database.
-----------------------------------
Telescope ID: 1
Name: Skinakas
Latitude :      35.21 deg N
Longitude:      24.90 deg E
Height:       1750.00 m            
-----------------------------------
2 associated parameter sets
-----------------------------------
Parameter set ID: 1
Active: False            
Constraints:
* Twilight
  - twilight: -12.0
* AirmassLimit
  - limit: 2.0
  - conversion: secz
* HourangleLimit
  - limit: 5.33
  - limit_lo: -5.33
* MoonDistance
  - limit: 10.0
-----------------------------------
Parameter set ID: 3
Active: True            
Constraints:
* Twilight
  - twilight: -12.0
* AirmassLimit
  - limit: 2.0
  - conversion: secz
* HourangleLimit
  - limit: 5.33
  - limit_lo: -5.33
* MoonDistance
  - limit: 5.0
-----------------------------------
Telescope ID: 2
Name: SAAO
Latitude :     -32.38 deg N
Longitude:      20.81 deg E
Height:       1798.00 m            
---------------------------------

### `FieldManager`

The `FieldManager` class provides the methods to add fields to and read them from the database.
Below, we describe the methods relevant for user interaction. Other methods are used by the survey planning software.

In [37]:
db_name = 'test_planner_temp.sqlite3'
manager = FieldManager(db_name)

#### Add fields

We can use the same method as in the `DBCreator` class to add fields: `add_fields()`, which works exactly the same way as shown above. The manager does not check whether or not a field with the same coordinates already exits in the database. We are not going to add more fields here.

#### Get fields

The method `get_fields()` provides various options to query fields that can all be combined. Without any parameters, the method returns all stored fields:

In [38]:
fields = manager.get_fields()
len(fields)

1644

With can specify the telescope to get all fields associated with that telescope:

In [39]:
DataFrame(manager.get_fields(telescope='Skinakas'))

,field_id,fov,center_ra,center_dec,tilt,telescope,active,jd_first,jd_next,nobs_tot,nobs_done,nobs_pending
0,1,0.087266,0.000000,0.043633,0.0,Skinakas,1,None,None,3,0,3
1,2,0.087266,0.070598,0.043633,0.0,Skinakas,1,None,None,1,0,1
2,3,0.087266,0.141195,0.043633,0.0,Skinakas,1,None,None,1,0,1
3,4,0.087266,0.211793,0.043633,0.0,Skinakas,1,None,None,1,0,1
4,5,0.087266,0.282390,0.043633,0.0,Skinakas,1,None,None,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1092,1093,0.087266,3.769911,1.483530,0.0,Skinakas,1,None,None,1,0,1
1093,1094,0.087266,4.398230,1.483530,0.0,Skinakas,1,None,None,1,0,1
1094,1095,0.087266,2.692794,1.527163,0.0,Skinakas,1,None,None,1,0,1
1095,1096,0.087266,3.590392,1.527163,0.0,Skinakas,1,None,None,1,0,1


We can ask specifically for fields that have been observed at least once by setting `observed=True` or for fields that have never been observed by setting `observed=False`). E.g.:

In [16]:
DataFrame(manager.get_fields(observed=True))

""


Similarly, we can ask specifically for fields that have pending observations by setting `pending=True` or for fields that do not have any pending observations by setting `pending=False`). E.g.:

In [17]:
DataFrame(manager.get_fields(pending=True))

,field_id,fov,center_ra,center_dec,tilt,telescope,active,jd_first,jd_next,nobs_tot,nobs_done,nobs_pending
0,1,0.087266,0.000000,0.043633,0.0,Skinakas,1,None,None,1,0,1
1,2,0.087266,0.070598,0.043633,0.0,Skinakas,1,None,None,1,0,1
2,3,0.087266,0.141195,0.043633,0.0,Skinakas,1,None,None,1,0,1
3,4,0.087266,0.211793,0.043633,0.0,Skinakas,1,None,None,1,0,1
4,5,0.087266,0.282390,0.043633,0.0,Skinakas,1,None,None,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1639,1640,0.122173,5.750712,-0.052360,0.0,SAAO,1,None,None,1,0,1
1640,1641,0.122173,5.857207,-0.052360,0.0,SAAO,1,None,None,1,0,1
1641,1642,0.122173,5.963701,-0.052360,0.0,SAAO,1,None,None,1,0,1
1642,1643,0.122173,6.070196,-0.052360,0.0,SAAO,1,None,None,1,0,1


By default we search for fields that are active. We can specifically ask for inactive fields by setting `active=False`:

In [18]:
DataFrame(manager.get_fields(active=False))

""


The method provides two more options that are not of particular use to the user. They are needed for the survey planning software.

<div class="alert alert-block alert-warning">

- Add options to query by coordinate: (a) fields that include the coordinates, (b) fields in a circle around the coordinates.
    
</div>

#### Deactivate fields

<div class="alert alert-block alert-warning">
    
- Add method to deactivate fields.

</div>

#### Print infos about fields

We have the option to get info about all fields or for fields associated with a specific telescope by specifying the telescope name. By default only active fields are considered. We can include active and inactive fields by setting `active=None` or get infos only about inactive fields by setting `active=False`.

In [19]:
manager.info()

============== FIELDS =============
All active fields in the database
-----------------------------------
Total number of fields:        1644
Pending fields:                1644
Pending observations:          1644
Finished fields:                  0
Finished observations:            0
-----------------------------------



In [20]:
manager.info(telescope='Skinakas')

============== FIELDS =============
Active fields associated with telescope 'Skinakas'
-----------------------------------
Total number of fields:        1097
Pending fields:                1097
Pending observations:          1097
Finished fields:                  0
Finished observations:            0
-----------------------------------



In [21]:
manager.info(active=False)

============== FIELDS =============
All inactive fields in the database
-----------------------------------
Total number of fields:           0
-----------------------------------



### `GuidestarManager`

The `GuidestarManager` class provides the methods to add fields to and read them from the database.
Below, we describe the methods relevant for user interaction. Other methods are used by the survey planning software.

In [22]:
db_name = 'test_planner_temp.sqlite3'
manager = GuidestarManager(db_name)

#### Add guidestars

We can use the same method as in the `DBCreator` class to add guidestars: `add_guidestars()`, which works exactly the same way as shown above. We are not going to add more guidestars here.

#### Deactivate guidestars

If a guidestar does not work well for guideing for whatever reason, we can deactivate it. We can add a single guidestar ID or multiple as a list to the `deactivate()` method. Note that the following function does not check whether these ID exists or whether these guidestars are active in the first place.

In [23]:
manager.deactivate([1, 1643])

Deactivated 2 guidestars.


#### Get guidestars

We can use the `get_guidestars()` method to get a list of guidestars. By default, we get all guidestars. We can explicitly query the guidestars for a specific field through the field ID. By default we query only guidestars that are active. We can also query only inactive guidestars by setting `active=False` or include active and inactive guidestars by setting `active=None`. As a result we get a list. Each item is a tuple that describes one guidestar, giving the guidestar ID, associated field ID, guidestar right ascension and declination, and the active flag.

In [25]:
DataFrame(manager.get_guidestars(field_id=1, active=None))

,guidestar_id,field_id,ra,dec,active
0,1,1,0.0,0.043633,0


#### Get fields missing guidestars

This method returns the IDs of fields that have no associated guidestars:

In [27]:
manager.get_fields_missing_guidestar()

{'none': [1643, 1644], 'inactive': [1]}

#### Print infos about guidestars

The `info()` method prints information about the guidestars stored in the database.

In [30]:
manager.info()

============ GUIDESTARS ===========
Total:                         1642
Active:                        1641
Inactive:                         1
Fields w. guidestars:          1641
Fields w/o guidestar:             2
Fields w/o active guidestar:      1
-----------------------------------



We can also ask about a specific field by setting its ID:

In [31]:
manager.info(field_id=1)

============ GUIDESTARS ===========
Associated with field ID 1
Total:                            1
Active:                           0
Inactive:                         1
-----------------------------------



### `ObservationManager`: manage observations

The `ObservationManager` class provides the methods to add, manage, and read observations to/in/from the database.
Below, we describe the methods relevant for user interaction. Other methods are used by the survey planning software.

In [32]:
db_name = 'test_planner_temp.sqlite3'
manager = ObservationManager(db_name)

#### Add observations

We can use the same method as in the `DBCreator` class to add observations: `add_observations()`. If we do not specify any field, we can add the same observation to all fields:

In [33]:
manager.add_observations(30., 2, 'r')

1 observation(s) with the same parameters already exist in data base. 0 out of those are finished. Add new observation anyway? (y/n, 'ALL' to add all following without asking, or 'NONE' to skip all existing observations that have not been finished). NONE


0 observations added to data base.


Alternatively, we can add the same type of observation to all fields associated with a specific telescope, e.g. by setting `telescope='Skinakas'`:

In [34]:
manager.add_observations(30., 2, 'r', telescope='Skinakas')

1 observation(s) with the same parameters already exist in data base. 0 out of those are finished. Add new observation anyway? (y/n, 'ALL' to add all following without asking, or 'NONE' to skip all existing observations that have not been finished). NONE


0 observations added to data base.


We can also add observations to a specific field by setting the `field_id`:

In [35]:
manager.add_observations(60, 2, 'r', field_id=1)
manager.add_observations(60, 2, 'i', field_id=1)

1 observations added to data base.


Filter 'i does not exist. Add it to data base? (y/n) y


1 observations added to data base.


#### Get observations

We can use the `get_guidestars()` method to get a list of guidestars. By default, we get all guidestars. We can explicitly query the guidestars for a specific field through the field ID. By default we query only guidestars that are active. We can also query only inactive guidestars by setting `active=False` or include active and inactive guidestars by setting `active=None`. As a result we get a list. Each item is a tuple that describes one guidestar, giving the guidestar ID, associated field ID, guidestar right ascension and declination, and the active flag.

In [36]:
DataFrame(manager.get_observations(field_id=None, active=None))

,observation_id,field_id,exposure,repetitions,filter,done,scheduled,active,date_done
0,1,1,30.0,2,r,0,0,1,None
1,2,2,30.0,2,r,0,0,1,None
2,3,3,30.0,2,r,0,0,1,None
3,4,4,30.0,2,r,0,0,1,None
4,5,5,30.0,2,r,0,0,1,None
...,...,...,...,...,...,...,...,...,...
1641,1642,1642,30.0,2,r,0,0,1,None
1642,1643,1643,30.0,2,r,0,0,1,None
1643,1644,1644,30.0,2,r,0,0,1,None
1644,1645,1,60.0,2,r,0,0,1,None
